In [2]:
import sys
import numpy as np
import pandas as pd
import html
import re
import json
import pickle
import emoji
import ast

import spacy
from spacy.lang.id import Indonesian

nlp = Indonesian()  # use directly
stopwords = spacy.lang.id.stop_words.STOP_WORDS
stopwords.add("nya")
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_colwidth', -1)

In [3]:
data = pd.read_csv("all.csv")
data.head()

,id,user_name,created_at,text,trendings
0,1123983568500400129,me_idy,2019-05-02 16:12:19,"#SemogaPuasaTahunIni nggak sendiri lagi.. Tapi, itu cuma ekspetasi 😕 Kenyataannya... Masih JOMBLO!! https://t.co/3TcbJrO8M4",#SemogaPuasaTahunIni
1,1123983419283845125,robbseptiants,2019-05-02 16:11:44,"#SemogaPuasaTahunIni bisa tahan godaan dari orng², liar, nakal, brutal membuat orang menjadi gempar.",#SemogaPuasaTahunIni
2,1123983321787195393,natamiayunda,2019-05-02 16:11:21,RT @gryprmna: #SemogaPuasaTahunIni kaga mikirin people we can't have terooos,#SemogaPuasaTahunIni
3,1123983291210539008,AlFathirAU__,2019-05-02 16:11:13,"#SemogaPuasaTahunIni disehatkan mental dan jasmani, berubah sedikit ke arah yang lebih baik. Aaminn",#SemogaPuasaTahunIni
4,1123983249364090881,Feriikrnwn_,2019-05-02 16:11:03,#SemogaPuasaTahunIni bukber nya ga wacana semuaa,#SemogaPuasaTahunIni


In [4]:
#### PREPROCESSING

# #Repeated Word
# def repeated(token):
#     repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
#     match = r'\1\2\3'
#     def replace(old_word):
#         if old_word in bag:
# #         ps = PorterStemmer()
# #         ps.stem(old_word)
# #         if wordnet.synsets(old_word):
# #         if old_word in words.words():
#             return old_word
#         new_word = repeat_pattern.sub(match, old_word)
#         return replace(new_word) if new_word != old_word else new_word
    
#     correct = [replace(word) for word in token]
#     return correct   

# #Akronim
def slang(tokenized_sentence):
    slang_word_dict = json.loads(open("slang_word_dict.txt", 'r').read())
#     print('tes')
    for index in range(len(tokenized_sentence)):
        for key, value in slang_word_dict.items():
            for v in value:
#                 print(tokenized_sentence[index],v)
                if tokenized_sentence[index] == v:
                    tokenized_sentence[index] = key
                else:
                    continue           
    return " ".join(tokenized_sentence)

def stopword(teks):
    clean = []
    for i in teks:
        if i not in stopwords:
            clean.append(i)

    return clean

In [5]:
def preprocessing(text):
    clean = text.lower() #lowercase
    clean = html.unescape(clean) #html tag
    clean = re.sub(r"rt @[\w_]+:", " ", clean) #no RT
    clean = re.sub("@[A-Za-z0-9]+", "", clean) #no Mention
    clean = re.sub("[0-9]", "", clean) #no Number
    clean = re.sub(r"http\S+", "", clean) #no HTML
    clean = emoji.get_emoji_regexp().sub("", clean) #EMOJI remover
    clean = " ".join(re.findall("[#a-zA-Z]{3,}", clean)) #Puntc
    clean = [token.text for token in nlp(clean)] #Token
    #repeated
    clean = slang(clean)#slang word
    clean = [token.lemma_ for token in nlp(clean)] #Lemma
    clean = stopword(clean) #Stopword
    
    return clean

In [6]:
data['clean'] = data['text'].apply(preprocessing)
data.head()

,id,user_name,created_at,text,trendings,clean
0,1123983568500400129,me_idy,2019-05-02 16:12:19,"#SemogaPuasaTahunIni nggak sendiri lagi.. Tapi, itu cuma ekspetasi 😕 Kenyataannya... Masih JOMBLO!! https://t.co/3TcbJrO8M4",#SemogaPuasaTahunIni,"[#semogapuasatahunini, ekspetasi, nyata, jomblo]"
1,1123983419283845125,robbseptiants,2019-05-02 16:11:44,"#SemogaPuasaTahunIni bisa tahan godaan dari orng², liar, nakal, brutal membuat orang menjadi gempar.",#SemogaPuasaTahunIni,"[#semogapuasatahunini, tahan, goda, orang, liar, nakal, brutal, orang, gempar]"
2,1123983321787195393,natamiayunda,2019-05-02 16:11:21,RT @gryprmna: #SemogaPuasaTahunIni kaga mikirin people we can't have terooos,#SemogaPuasaTahunIni,"[#semogapuasatahunini, pikir, orang, terooos]"
3,1123983291210539008,AlFathirAU__,2019-05-02 16:11:13,"#SemogaPuasaTahunIni disehatkan mental dan jasmani, berubah sedikit ke arah yang lebih baik. Aaminn",#SemogaPuasaTahunIni,"[#semogapuasatahunini, disehatkan, mental, jasmani, ubah, arah, aaminn]"
4,1123983249364090881,Feriikrnwn_,2019-05-02 16:11:03,#SemogaPuasaTahunIni bukber nya ga wacana semuaa,#SemogaPuasaTahunIni,"[#semogapuasatahunini, bukber, wacana, semuaa]"


In [7]:
df = pd.DataFrame(data=data)
df.to_csv("C:/Users/USER/Downloads/tegar nitip/SMT 8/AMS/FP/clean.csv", sep=',',index=False)